In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/endpoint/detay.txt
/kaggle/input/endpoint/untitled.py


In [2]:
# Gerekli Kütüphaneleri Yükleyelim
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install --upgrade nbformat
!pip install Flask


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import datetime
from pycoingecko import CoinGeckoAPI
from flask import Flask, jsonify

In [4]:
# CoinGecko API Bağlantısı
cg = CoinGeckoAPI()


In [5]:
# Flask Uygulaması Başlatma
app = Flask(__name__)

In [6]:
# Bitcoin ve Ethereum'un fiyat verilerini çekme fonksiyonları
@app.route('/bitcoin', methods=['GET'])
def get_bitcoin_price():
    bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=1)
    bitcoin_price_data = bitcoin_data['prices']
    bitcoin_df = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])
    bitcoin_df['date'] = bitcoin_df['TimeStamp'].apply(lambda d: datetime.datetime.utcfromtimestamp(d / 1000.0).strftime('%Y-%m-%d %H:%M:%S'))
    
    latest_price = bitcoin_df.iloc[-1]
    price = latest_price['Price']
    timestamp = latest_price['date']

    return jsonify({
        'coin': 'Bitcoin',
        'price': price,
        'timestamp': timestamp
    })

@app.route('/ethereum', methods=['GET'])
def get_ethereum_price():
    ethereum_data = cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='eur', days=1)
    ethereum_price_data = ethereum_data['prices']
    ethereum_df = pd.DataFrame(ethereum_price_data, columns=['TimeStamp', 'Price'])
    ethereum_df['date'] = ethereum_df['TimeStamp'].apply(lambda d: datetime.datetime.utcfromtimestamp(d / 1000.0).strftime('%Y-%m-%d %H:%M:%S'))
    
    latest_price = ethereum_df.iloc[-1]
    price = latest_price['Price']
    timestamp = latest_price['date']

    return jsonify({
        'coin': 'Ethereum',
        'price': price,
        'timestamp': timestamp
    })
    

In [7]:
# Coin verilerini hazırlama ve görselleştirme
# Bitcoin ve Ethereum verilerini çekme
bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=365)
ethereum_data = cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='eur', days=30)

In [8]:
# Verileri DataFrame'e dönüştürme
bitcoin_price_data = bitcoin_data['prices']
ethereum_price_data = ethereum_data['prices']

bitcoin_df = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])
ethereum_df = pd.DataFrame(ethereum_price_data, columns=['TimeStamp', 'Price'])


In [9]:
# Veri Temizliği ve Zenginleştirme
# Boş Verileri Kontrol Etme ve Temizleme
bitcoin_df.dropna(subset=['Price'], inplace=True)
ethereum_df.dropna(subset=['Price'], inplace=True)

In [10]:
# Anomalileri ve Aykırı Değerleri Kontrol Etme
bitcoin_df = bitcoin_df[bitcoin_df['Price'] > 0]
ethereum_df = ethereum_df[ethereum_df['Price'] > 0]

In [11]:
# Zaman Damgasını Tarihe Çevirme
bitcoin_df['date'] = bitcoin_df['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d / 1000.0))
ethereum_df['date'] = ethereum_df['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d / 1000.0))

In [12]:
# Candlestick Verilerini Hazırlama
bitcoin_candlestick_data = bitcoin_df.groupby('date', as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
ethereum_candlestick_data = ethereum_df.groupby('date', as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})


In [13]:
# Veriyi Normalleştirme (Opsiyonel)
bitcoin_df['Price_Normalized'] = (bitcoin_df['Price'] - bitcoin_df['Price'].min()) / (bitcoin_df['Price'].max() - bitcoin_df['Price'].min())
ethereum_df['Price_Normalized'] = (ethereum_df['Price'] - ethereum_df['Price'].min()) / (ethereum_df['Price'].max() - ethereum_df['Price'].min())


In [14]:
# Görselleştirme - Mum Grafik
fig = go.Figure()


In [15]:
# Bitcoin Grafiğini ekle
fig.add_trace(go.Candlestick(
    x=bitcoin_candlestick_data['date'],
    open=bitcoin_candlestick_data['Price']['first'],
    high=bitcoin_candlestick_data['Price']['max'],
    low=bitcoin_candlestick_data['Price']['min'],
    close=bitcoin_candlestick_data['Price']['last'],
    name="Bitcoin (USD)"
))

In [16]:
# Ethereum Grafiğini ekle
fig.add_trace(go.Candlestick(
    x=ethereum_candlestick_data['date'],
    open=ethereum_candlestick_data['Price']['first'],
    high=ethereum_candlestick_data['Price']['max'],
    low=ethereum_candlestick_data['Price']['min'],
    close=ethereum_candlestick_data['Price']['last'],
    name="Ethereum (EUR)"
))


In [17]:
# Grafik düzenlemeleri
fig.update_layout(
    title="Bitcoin ve Ethereum Fiyat Hareketleri",
    xaxis_rangeslider_visible=True,
    xaxis_title="Tarih",
    yaxis_title="Fiyat",
    xaxis=dict(tickformat='%Y-%m-%d'),
    legend=dict(x=0.01, y=0.99, traceorder='normal', orientation='h')
)

In [18]:
# Grafiği gösterme
fig.show()